In [1]:
import sys; sys.path.insert(0, '../../')
from definitions import *
from notebooks.nb_imports import *

In [3]:
from experiments.utils import create_run_frame
from src.visualization.critical_difference import plot_cd_diagram, wilcoxon_holm

# Evaluating the best signature pipeline model
Here we give the process for extracting the best signature pipeline model, as defined in the paper. The code is a little different to that seen in `extract_and_analyse.ipynb` since we are looking for the maximum over a gridsearch of parameters, rather than comparing between each run of the model.

## 1. Extract the run information

In [ ]:
df = create_run_frame(RESULTS_DIR + '/best_rf')
df.head()

## 2. Create the transform_id column

In [6]:
# Extract best
df['transform_id'] = df['depth'].astype(str) + df['window']
df = df.loc[~df[['ds_name', 'transform_id']].duplicated().values]

## 3. Maximise over the oob_score
We choose the best signature depth and dyadic depth by selecting the model that maximised the oob score on the training set. 

In [8]:
oob_max = df.groupby(['ds_name']).apply(lambda x: x.loc[x['oob_score.train'].idxmax()])
best_test = oob_max['acc.test']

# Lets call it the signature pipeline model and fix a naming inconsistency
best_test.name = 'Signature pipeline'
best_test.rename({'PhonemeSpectra': 'Phoneme'}, inplace=True)

## 4. Compare with the pre-existing benchmarks
Check the final table in the paper for the pre-existing benchmarks. `best_test` now contains the score of the signature pipeline model on the datasets we tested on.

In [10]:
best_test

ds_name
ERing             0.937037
Epilepsy          0.971014
JapaneseVowels    0.959459
LSST              0.523520
Libras            0.905556
NATOPS            0.877778
Name: Signature pipeline, dtype: float64

## 5. Further evaluation

We can now collate the results and perform signed ranks tests, such as is described in the `extract_and_analyse.ipynb` file. We leave this up to the user. 

## 6. Getting the hyperparameters
In case you also want the best hyperparamters for each model, the following code gives an example of how to extract them. 

In [11]:
# Setup dfs
params_used = oob_max[['depth', 'window', 'param_grid', 'training_time']]
rf_params = pd.DataFrame(index=params_used.index, columns=['rf__max_depth', 'rf__n_estimators'])

In [19]:
new_window = params_used['window'].apply(lambda x: eval(x)['py/tuple'][-1]['depth'])
params_used['window'] = new_window

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
# Open the param_grid dict into columns for each parameter
for idx in params_used.index:
    rf_grid = params_used.loc[idx, 'param_grid']
    rf_params.loc[idx, 'rf__max_depth'] = rf_grid['classifier__max_depth']
    rf_params.loc[idx, 'rf__n_estimators'] = rf_grid['classifier__n_estimators']
all_params = pd.concat([params_used.drop('param_grid', axis=1), rf_params], axis=1).fillna('None')
all_params['training_time'] = all_params['training_time'].round(1)
all_params = all_params[['depth', 'window', 'rf__max_depth', 'rf__n_estimators', 'training_time']]

In [21]:
all_params

,depth,window,rf__max_depth,rf__n_estimators,training_time
ds_name,,,,,
ERing,3,3,45,50,2.3
Epilepsy,5,2,16,50,22.2
JapaneseVowels,3,2,60,50,10.0
LSST,2,2,12,50,15.1
Libras,6,2,80,50,13.5
NATOPS,2,4,60,50,6.4
